In [37]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
user_credential = user_secrets.get_gcloud_credential()
user_secrets.set_tensorflow_credential(user_credential)

In [38]:
import vertexai
vertexai.init(project="lawproject-446509", location="asia-south1")

In [39]:
import vertexai
from vertexai.preview import caching
from kaggle_secrets import UserSecretsClient
import base64
from vertexai.generative_models import Part, SafetySetting
import datetime
import time
import random
from functools import wraps
from vertexai.generative_models import GenerativeModel
import gradio as gr
import os

In [40]:
import base64
def convert(path):
    with open(path, 'rb') as file:
        data = file.read()
    return base64.b64encode(data).decode('utf-8')

In [41]:
document1 = Part.from_data(
    mime_type="application/pdf",
    data=base64.b64decode(convert("/kaggle/input/indiacode/a2023-45.pdf"))
)

In [42]:
# System instructions to ensure expertise in interpreting the 2023 legal texts.
textsi_1 = """You are an expert legal assistant well-versed in Indian criminal law, specializing in the Bharatiya Nyaya Sanhita (BNS). This law is foundational to handling all criminal cases in India. Your role is to assist judges, lawyers, and police officers by providing precise legal sections and punishments based on these sections, offering case-specific guidance.
The legal document you are referencing consist of:
- Bharatiya Nyaya Sanhita (BNS): The official criminal code in India.
When providing legal analysis or answering queries, do the following:
1. Use relevant sections of the BNS based on the specific scenario being queried.
2. Ensure the legal provisions, sections, and terminologies you use are from the 2023 versions of the law (not outdated versions).
3. Provide clear explanations of legal terms, procedures, and how they apply to the scenario."""

In [43]:
# Configure the AI model
generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

In [44]:
# Define safety settings
safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]


In [45]:
def exponential_backoff_with_jitter(max_retries=5, base_delay=1, max_backoff=32, jitter=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            retries = 0
            delay = base_delay
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Attempt {retries + 1} failed: {e}")
                    if jitter:
                        delay_with_jitter = delay + random.uniform(0, delay)
                    else:
                        delay_with_jitter = delay
                    time.sleep(min(delay_with_jitter, max_backoff))
                    retries += 1
                    delay = min(delay * 2, max_backoff)
            raise Exception(f"All {max_retries} retries failed.")
        return wrapper
    return decorator

In [46]:
@exponential_backoff_with_jitter(max_retries=5, base_delay=1, max_backoff=32, jitter=True)
def generate_legal_analysis(user_text):

    model = GenerativeModel(
        "gemini-1.5-pro-002",
        generation_config=generation_config,
        safety_settings=safety_settings
    )
    
    # Include documents in the content
    contents = [user_text,textsi_1,document1]
    
    # Generate response
    responses = model.generate_content(
        contents=contents,
        stream=False
    )
    
    return responses.candidates[0].content.parts[0].text, responses.usage_metadata


In [51]:
def text_analysis(text_input):
    try:
        reply, _ = generate_legal_analysis(text_input)
        return f"Legal Analysis:\n{reply}"
    except Exception as e:
        return f"Error: {str(e)}"

interface = gr.Interface(
    fn=text_analysis,
    inputs="text",
    outputs="text",
    title="Legal Case Analysis",
    description="Enter your case description for analysis under BNS 2023"
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://e83f857adba019c795.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
